Name: Aritra Bandyopadhyay

Roll No: 2021CSB107

Subject: Artificial Intelligence Laboratory (CS 4271)

## Problem

A disaster has struck a futuristic smart city, and your task is to develop an AI-powered drone navigation system to rescue stranded survivors. The drone must autonomously navigate through the city, avoiding obstacles like collapsed buildings and fire zones while optimizing for the fastest route (equivalent to minimum energy consumption).

Your objective is to implement the A Search Algorithm* to guide the drone from its starting position (S) to rescue multiple survivors (G1, G2, ... , Gn) (intermediate states) and return to base (B) (treated as goal state) while minimizing energy consumption and ensuring safe traversal.

### Drone Movement and Cost Considerations:

a) The drone moves in 3D space:
* Up (x, y, z+1), Down (x, y, z-1),
* North (x-1, y, z), South (x+1, y, z),
* East (x, y+1, z), West (x, y-1, z).
* Diagonal movements (like flying at an angle) are not allowed for simplicity.

b) Energy Consumption Factor:
* Moving through clear space (0) costs 1 energy unit.
* Moving through fire zones (F) costs 3 energy units due to turbulence.
* Moving upwards (z+1) costs extra 2 energy units, while descending (z-1) costs 1 energy unit.

c) Objective:
* The drone must rescue all survivors before returning to base (B).
* The optimal path minimizes total energy consumption.
* If needed, the drone can recharge at (R), but stopping at a station adds a fixed time penalty.

In [26]:
import heapq
import math
from collections import deque

In [27]:

class DroneRescue:
    def __init__(self, grid, max_energy=100, recharge_penalty=5):
        """Initialize the drone rescue system with a 3D grid representation of the city"""
        self.grid = grid
        self.max_energy = max_energy
        self.recharge_penalty = recharge_penalty
        self.rows = len(grid[0])
        self.cols = len(grid[0][0])
        self.levels = len(grid)
        
        # Find start, base, survivors, and recharge stations
        self.start = None
        self.base = None
        self.survivors = []
        self.recharge_stations = []
        
        # Parse the grid
        for z in range(self.levels):
            for x in range(self.rows):
                for y in range(self.cols):
                    cell = grid[z][x][y]
                    pos = (x, y, z)
                    
                    if cell == 'S':
                        self.start = pos
                    elif cell == 'B':
                        self.base = pos
                    elif cell == 'R':
                        self.recharge_stations.append(pos)
                    elif cell == 'G' or (isinstance(cell, str) and cell.startswith('G')):
                        self.survivors.append(pos)
        
        print(f"Grid dimensions: {self.rows}x{self.cols}x{self.levels}")
        print(f"Start position: {self.start}")
        print(f"Base position: {self.base}")
        print(f"Survivors: {self.survivors}")
        print(f"Recharge stations: {self.recharge_stations}")
    
    def get_cell_type(self, pos):
        """Get the cell type at a given position"""
        x, y, z = pos
        return self.grid[z][x][y]
    
    def euclidean_distance(self, pos1, pos2):
        """Calculate 3D Euclidean distance heuristic between two positions"""
        x1, y1, z1 = pos1
        x2, y2, z2 = pos2
        return math.sqrt((x1 - x2)**2 + (y1 - y2)**2 + (z1 - z2)**2)
    
    def manhattan_distance(self, pos1, pos2):
        """Calculate 3D Manhattan distance heuristic between two positions"""
        x1, y1, z1 = pos1
        x2, y2, z2 = pos2
        return abs(x1 - x2) + abs(y1 - y2) + abs(z1 - z2)
    
    def get_neighbors(self, pos):
        """Get valid neighboring positions (Up, Down, North, South, East, West)"""
        x, y, z = pos
        possible_moves = [
            (x, y, z+1),  # Up
            (x, y, z-1),  # Down
            (x-1, y, z),  # North
            (x+1, y, z),  # South
            (x, y+1, z),  # East
            (x, y-1, z)   # West
        ]
        
        valid_neighbors = []
        for nx, ny, nz in possible_moves:
            # Check if within grid bounds
            if 0 <= nx < self.rows and 0 <= ny < self.cols and 0 <= nz < self.levels:
                # Check if not a building/obstacle
                cell = self.grid[nz][nx][ny]
                if cell != '1' and cell != 1:
                    valid_neighbors.append((nx, ny, nz))
        
        return valid_neighbors
    
    def get_move_cost(self, current, next_pos):
        """Calculate the energy cost of moving from current to next position"""
        _, _, z1 = current
        x2, y2, z2 = next_pos
        
        # Base cost for movement (1 energy unit)
        cost = 1
        
        # Additional cost for fire zone (3 energy units total)
        cell = self.grid[z2][x2][y2]
        if cell == 'F':
            cost += 2
        
        # Additional cost for vertical movement
        if z2 > z1:  # Moving up (3 energy units total)
            cost += 2
        
        return cost
    
    def a_star_search(self, start, goal, current_energy):
        """A* search algorithm implementation for finding energy-efficient path"""
        # Priority queue for A* search: (f_score, tiebreaker, pos, energy, path, g_score)
        counter = 0  # Tiebreaker for equal f-scores
        open_set = [(self.euclidean_distance(start, goal), counter, start, current_energy, [start], 0)]
        heapq.heapify(open_set)
        
        # To track visited states with their best energy
        closed_set = {}  # (position) -> best_energy_seen
        
        # For tracking algorithm progress
        expanded_nodes = 0
        
        while open_set:
            f, _, current, energy, path, g = heapq.heappop(open_set)
            expanded_nodes += 1
            
            # Print progress info
            # if expanded_nodes % 100 == 0 or expanded_nodes < 10:
            print(f"Expanded {expanded_nodes} nodes, queue size: {len(open_set)}")
            print(f"Current: {current}, f(n): {f:.3f}, g(n): {g:.3f}, h(n): {f-g:.3f}, energy: {energy}")
            
            # Goal check
            if current == goal:
                print(f"Path found! Expanded {expanded_nodes} nodes")
                return path, current_energy - energy, energy
            
            # Skip if we've seen this position with better energy
            if current in closed_set and closed_set[current] >= energy:
                continue
            closed_set[current] = energy
            
            # Check neighbors
            for neighbor in self.get_neighbors(current):
                move_cost = self.get_move_cost(current, neighbor)
                
                # Check if enough energy
                if energy < move_cost:
                    # Check if at a recharge station
                    if self.get_cell_type(current) == 'R':
                        # Recharge fully but incur time penalty
                        new_energy = self.max_energy - move_cost
                        new_g = g + move_cost + self.recharge_penalty
                        print(f"Recharging at {current}, new energy: {new_energy}")
                    else:
                        # Not enough energy and not at a recharge station
                        continue
                else:
                    new_energy = energy - move_cost
                    new_g = g + move_cost
                
                # Calculate heuristic and f-score
                new_h = self.euclidean_distance(neighbor, goal)
                new_f = new_g + new_h
                
                # Skip if we've seen this position with better energy
                if neighbor in closed_set and closed_set[neighbor] >= new_energy:
                    continue
                
                # New path to neighbor
                new_path = path + [neighbor]
                
                # Add to open set
                counter += 1
                heapq.heappush(open_set, (new_f, counter, neighbor, new_energy, new_path, new_g))
                
                # Print heuristic values for some neighbors (for clarity)
                if expanded_nodes < 5:
                    print(f"  Neighbor: {neighbor}, g: {new_g:.3f}, h: {new_h:.3f}, f: {new_f:.3f}, energy: {new_energy}")
        
        # No path found
        print(f"No path found after expanding {expanded_nodes} nodes!")
        return None, 0, 0
    
    def find_nearest_goal(self, current, goals):
        """Find the nearest goal from the current position"""
        min_dist = float('inf')
        nearest = None
        
        for goal in goals:
            dist = self.euclidean_distance(current, goal)
            if dist < min_dist:
                min_dist = dist
                nearest = goal
        
        return nearest
    
    def solve(self):
        """Solve the drone rescue mission using a greedy approach for multi-goal search"""
        if not self.start or not self.base or not self.survivors:
            print("Error: Missing start, base, or survivors!")
            return None, 0
        
        # Initialize
        current_pos = self.start
        current_energy = self.max_energy
        total_path = [current_pos]
        total_energy_used = 0
        survivors_to_visit = self.survivors.copy()
        
        # Visit each survivor in sequence (nearest-first greedy approach)
        while survivors_to_visit:
            # Find nearest survivor
            next_goal = self.find_nearest_goal(current_pos, survivors_to_visit)
            print(f"\nNavigating from {current_pos} to survivor at {next_goal}")
            
            # Find path to this survivor using A*
            path, energy_used, remaining_energy = self.a_star_search(
                current_pos, next_goal, current_energy
            )
            
            if not path:
                print(f"Warning: Cannot reach survivor at {next_goal}!")
                # Skip this survivor
                survivors_to_visit.remove(next_goal)
                continue
            
            # Update state
            current_pos = next_goal
            current_energy = remaining_energy
            total_path.extend(path[1:])  # Skip first position (already in path)
            total_energy_used += energy_used
            
            # Mark survivor as visited
            survivors_to_visit.remove(next_goal)
            
            print(f"Reached survivor at {current_pos}, remaining energy: {current_energy}")
        
        # Finally, return to base
        print(f"\nReturning to base at {self.base}")
        path, energy_used, _ = self.a_star_search(current_pos, self.base, current_energy)
        
        if not path:
            print("Error: Cannot return to base!")
            return total_path, total_energy_used
        
        # Complete the mission
        total_path.extend(path[1:])
        total_energy_used += energy_used
        
        print(f"\nMission completed!")
        print(f"Total energy used: {total_energy_used}")
        print(f"Final path length: {len(total_path)}")
        
        return total_path, total_energy_used
    
    def visualize_path(self, path):
        """Visualize the path with detailed step information"""
        if not path:
            print("No path to visualize!")
            return
        
        level_names = ["Ground", "Mid", "Sky"]
        total_energy_used = 0
        
        print("\nPath Visualization:")
        print("===================")
        
        for i, pos in enumerate(path):
            x, y, z = pos
            cell_type = self.get_cell_type(pos)
            level = level_names[z] if z < len(level_names) else f"Level {z}"
            
            # Print step information
            print(f"Step {i+1}: {level} level at ({x}, {y}), Cell: {cell_type}")
            
            # Print movement direction if not the first step
            if i > 0:
                prev_x, prev_y, prev_z = path[i-1]
                
                if x > prev_x:
                    direction = "South"
                elif x < prev_x:
                    direction = "North"
                elif y > prev_y:
                    direction = "East"
                elif y < prev_y:
                    direction = "West"
                elif z > prev_z:
                    direction = "Up"
                elif z < prev_z:
                    direction = "Down"
                else:
                    direction = "Same position"
                
                print(f"  Moved: {direction}")
                
                # Calculate movement cost
                move_cost = self.get_move_cost(path[i-1], pos)
                total_energy_used += move_cost
                print(f"  Energy used: {move_cost} (Total: {total_energy_used})")
                
                # Check for special cell types
                if cell_type == 'R':
                    print(f"  At recharge station")
                elif cell_type == 'F':
                    print(f"  Through fire zone")
                elif cell_type == 'G' or (isinstance(cell_type, str) and cell_type.startswith('G')):
                    print(f"  Rescued survivor")
                elif cell_type == 'B':
                    print(f"  Reached base")
        
        print("===================")
        print(f"Total energy used: {total_energy_used}")




In [28]:

def create_grid_from_description():
    """Create a 3D grid based on the problem description"""
    # Ground level (z=0)
    ground = [
        ['S', '0', '1', '0', '0'],
        ['0', 'F', '1', '1', 'G'],
        ['F', '0', '0', '0', 'F'],
        ['R', '1', '1', '1', '0'],
        ['B', '0', '1', '0', 'G']
    ]
    
    # Mid level (z=1)
    mid = [
        ['0', '0', '1', '0', '0'],
        ['0', '1', '1', '0', '0'],
        ['F', '0', '0', '0', '1'],
        ['0', '1', '1', '1', '0'],
        ['0', '0', '0', '0', '0']
    ]
    
    # Sky level (z=2)
    sky = [
        ['1', '0', '0', '0', '0'],
        ['0', 'F', '0', '1', '0'],
        ['0', '0', '1', '0', 'F'],
        ['0', '0', '0', '0', '0'],
        ['0', '1', '0', '0', '1']
    ]
    
    return [ground, mid, sky]


In [29]:

# Create grid
grid = create_grid_from_description()

# Initialize drone rescue system
drone = DroneRescue(grid, max_energy=25)

# Solve and visualize
path, energy_used = drone.solve()
if path:
    drone.visualize_path(path)


Grid dimensions: 5x5x3
Start position: (0, 0, 0)
Base position: (4, 0, 0)
Survivors: [(1, 4, 0), (4, 4, 0)]
Recharge stations: [(3, 0, 0)]

Navigating from (0, 0, 0) to survivor at (1, 4, 0)
Expanded 1 nodes, queue size: 0
Current: (0, 0, 0), f(n): 4.123, g(n): 0.000, h(n): 4.123, energy: 25
  Neighbor: (0, 0, 1), g: 3.000, h: 4.243, f: 7.243, energy: 22
  Neighbor: (1, 0, 0), g: 1.000, h: 4.000, f: 5.000, energy: 24
  Neighbor: (0, 1, 0), g: 1.000, h: 3.162, f: 4.162, energy: 24
Expanded 2 nodes, queue size: 2
Current: (0, 1, 0), f(n): 4.162, g(n): 1.000, h(n): 3.162, energy: 24
  Neighbor: (0, 1, 1), g: 4.000, h: 3.317, f: 7.317, energy: 21
  Neighbor: (1, 1, 0), g: 4.000, h: 3.000, f: 7.000, energy: 21
Expanded 3 nodes, queue size: 3
Current: (1, 0, 0), f(n): 5.000, g(n): 1.000, h(n): 4.000, energy: 24
  Neighbor: (1, 0, 1), g: 4.000, h: 4.123, f: 8.123, energy: 21
  Neighbor: (2, 0, 0), g: 4.000, h: 4.123, f: 8.123, energy: 21
  Neighbor: (1, 1, 0), g: 4.000, h: 3.000, f: 7.000, en

In [30]:
# Test Case 1: Different obstacle configuration
grid1 = [
    [   # Ground level
        ['S', '0', '1', '0', 'G'],
        ['0', 'F', '1', '1', '0'],
        ['F', '0', '0', '0', 'F'],
        ['R', '1', '1', '1', '0'],
        ['B', '0', '1', '0', '0']
    ],
    [   # Mid level
        ['0', '0', '1', '0', '0'],
        ['0', '1', '1', '0', '0'],
        ['F', '0', '0', '0', '1'],
        ['0', '1', '1', '1', '0'],
        ['0', '0', '0', '0', '0']
    ],
    [   # Sky level
        ['1', '0', '0', '0', '0'],
        ['0', 'F', '0', '1', '0'],
        ['0', '0', '1', '0', 'F'],
        ['0', '0', '0', '0', '0'],
        ['0', '1', '0', '0', '1']
    ]
]

# Test Case 2: Multiple goals
grid2 = [
    [   # Ground level
        ['S', '0', '1', '0', 'G'],
        ['0', 'F', '1', '1', 'G'],
        ['F', '0', '0', '0', 'F'],
        ['R', '1', '1', '1', '0'],
        ['B', '0', '1', '0', 'G']
    ],
    [   # Mid level
        ['0', '0', '1', '0', '0'],
        ['0', '1', '1', '0', '0'],
        ['F', '0', '0', '0', '1'],
        ['0', '1', '1', '1', '0'],
        ['0', '0', '0', '0', '0']
    ],
    [   # Sky level
        ['1', '0', '0', '0', '0'],
        ['0', 'F', '0', '1', '0'],
        ['0', '0', '1', '0', 'F'],
        ['0', '0', '0', '0', '0'],
        ['0', '1', '0', '0', '1']
    ]
]

# Test Case 3: Different start position
grid3 = [
    [   # Ground level
        ['0', '0', '1', '0', '0'],
        ['0', 'F', '1', '1', 'G'],
        ['F', '0', '0', '0', 'F'],
        ['R', '1', '1', '1', '0'],
        ['B', 'S', '1', '0', 'G']
    ],
    [   # Mid level
        ['0', '0', '1', '0', '0'],
        ['0', '1', '1', '0', '0'],
        ['F', '0', '0', '0', '1'],
        ['0', '1', '1', '1', '0'],
        ['0', '0', '0', '0', '0']
    ],
    [   # Sky level
        ['1', '0', '0', '0', '0'],
        ['0', 'F', '0', '1', '0'],
        ['0', '0', '1', '0', 'F'],
        ['0', '0', '0', '0', '0'],
        ['0', '1', '0', '0', '1']
    ]
]

# Run test cases
for i, grid in enumerate([grid1, grid2, grid3], 1):
    print(f"\nRunning Test Case {i}")
    drone = DroneRescue(grid, max_energy=100)
    path, energy_used = drone.solve()
    if path:
        drone.visualize_path(path)



Running Test Case 1
Grid dimensions: 5x5x3
Start position: (0, 0, 0)
Base position: (4, 0, 0)
Survivors: [(0, 4, 0)]
Recharge stations: [(3, 0, 0)]

Navigating from (0, 0, 0) to survivor at (0, 4, 0)
Expanded 1 nodes, queue size: 0
Current: (0, 0, 0), f(n): 4.000, g(n): 0.000, h(n): 4.000, energy: 100
  Neighbor: (0, 0, 1), g: 3.000, h: 4.123, f: 7.123, energy: 97
  Neighbor: (1, 0, 0), g: 1.000, h: 4.123, f: 5.123, energy: 99
  Neighbor: (0, 1, 0), g: 1.000, h: 3.000, f: 4.000, energy: 99
Expanded 2 nodes, queue size: 2
Current: (0, 1, 0), f(n): 4.000, g(n): 1.000, h(n): 3.000, energy: 99
  Neighbor: (0, 1, 1), g: 4.000, h: 3.162, f: 7.162, energy: 96
  Neighbor: (1, 1, 0), g: 4.000, h: 3.162, f: 7.162, energy: 96
Expanded 3 nodes, queue size: 3
Current: (1, 0, 0), f(n): 5.123, g(n): 1.000, h(n): 4.123, energy: 99
  Neighbor: (1, 0, 1), g: 4.000, h: 4.243, f: 8.243, energy: 96
  Neighbor: (2, 0, 0), g: 4.000, h: 4.472, f: 8.472, energy: 96
  Neighbor: (1, 1, 0), g: 4.000, h: 3.162, f

In [33]:

def create_grid_from_description():
    """Create a 3D grid based on the problem description"""
    # Ground level (z=0)
    ground = [
        ['0', '0', '1', 'S', '0'],
        ['0', 'F', '1', '1', 'G'],
        ['F', '0', '0', '0', 'F'],
        ['R', '1', '1', '1', '0'],
        ['B', '0', '1', '0', 'G']
    ]
    
    # Mid level (z=1)
    mid = [
        ['0', '0', '1', '0', '0'],
        ['0', '1', '1', '0', '0'],
        ['F', '0', '0', '0', '1'],
        ['0', '1', '1', '1', '0'],
        ['0', '0', '0', '0', '0']
    ]
    
    # Sky level (z=2)
    sky = [
        ['1', '0', '0', '0', '0'],
        ['0', 'F', '0', '1', '0'],
        ['0', '0', '1', '0', 'F'],
        ['0', '0', '0', '0', '0'],
        ['0', '1', '0', '0', '1']
    ]
    
    return [ground, mid, sky]


In [34]:

# Create grid
grid = create_grid_from_description()

# Initialize drone rescue system
drone = DroneRescue(grid, max_energy=25)

# Solve and visualize
path, energy_used = drone.solve()
if path:
    drone.visualize_path(path)


Grid dimensions: 5x5x3
Start position: (0, 3, 0)
Base position: (4, 0, 0)
Survivors: [(1, 4, 0), (4, 4, 0)]
Recharge stations: [(3, 0, 0)]

Navigating from (0, 3, 0) to survivor at (1, 4, 0)
Expanded 1 nodes, queue size: 0
Current: (0, 3, 0), f(n): 1.414, g(n): 0.000, h(n): 1.414, energy: 25
  Neighbor: (0, 3, 1), g: 3.000, h: 1.732, f: 4.732, energy: 22
  Neighbor: (0, 4, 0), g: 1.000, h: 1.000, f: 2.000, energy: 24
Expanded 2 nodes, queue size: 1
Current: (0, 4, 0), f(n): 2.000, g(n): 1.000, h(n): 1.000, energy: 24
  Neighbor: (0, 4, 1), g: 4.000, h: 1.414, f: 5.414, energy: 21
  Neighbor: (1, 4, 0), g: 2.000, h: 0.000, f: 2.000, energy: 23
Expanded 3 nodes, queue size: 2
Current: (1, 4, 0), f(n): 2.000, g(n): 2.000, h(n): 0.000, energy: 23
Path found! Expanded 3 nodes
Reached survivor at (1, 4, 0), remaining energy: 23

Navigating from (1, 4, 0) to survivor at (4, 4, 0)
Expanded 1 nodes, queue size: 0
Current: (1, 4, 0), f(n): 3.000, g(n): 0.000, h(n): 3.000, energy: 23
  Neighbor: 